In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import warnings
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')

In [2]:
# Load Mosaik data
csv_file = "path_to_mosaik_data.csv"
mosaik_data1 = pd.read_csv("files/[RAW] Mosaiks_features_1.csv")
mosaik_data2 = pd.read_csv("files/[RAW] Mosaiks_features_2.csv")
mosaik_merged = pd.concat([mosaik_data1, mosaik_data2])

# Load Shrid Polygon data
csv_file = "path_to_polygon_data.csv"
polygon_data1 = pd.read_csv("files/coords_inside_1.csv")
polygon_data2 = pd.read_csv("files/coords_inside_2.csv")
polygon_merged = pd.concat([polygon_data1, polygon_data2])

In [3]:
# Merge datasets on 'shrid'
data = pd.merge(polygon_merged, mosaik_merged, on=['Lon', 'Lat'], how='inner')

# Save the merged dataset
output_file = 'output/merged_dataset.csv'
data.to_csv(output_file, index=False)
print(f"Merged data saved to {output_file}")

Merged data saved to output/merged_dataset.csv


In [6]:
# data = pd.read_csv("output/merged_dataset.csv")

In [ ]:
# Rename columns with MOSAIKS data
for i, col in enumerate(data.columns[4:], start=1):
    data.rename(columns={col: f"Mosaik_{i}"}, inplace=True)
data.to_csv("output/renamed_features.csv", index=False)
data.head(2)

In [ ]:
# Transform MOSAIKS data using PCA
mosaik_features = data.iloc[:, 4:]
pca = PCA(n_components=1)
data['PCA_1'] = pca.fit_transform(mosaik_features)

In [16]:
# Read polygon data containing the village boundaries
polygon_data = pd.read_csv("files/Mosaik_Shrid_coordinates_Data_Davide_Rahul_Ashesh_Leonard_2024 - Mosaik_Shrid_coordinates_Data_Davide_Rahul_Ashesh_Leonard_2024.csv")

In [38]:
# Draw heatmaps for each village
for i in range(1, len(polygon_data) + 1):
	selected_village = i

	selected_data = data[data['Unnamed: 0'] == selected_village]
	selected_polygon = Polygon(eval(polygon_data.iloc[selected_village - 1]['polygon_coordinates']))

	x_min, y_min, x_max, y_max = selected_polygon.bounds
	map_center = [(y_min + y_max) / 2, (x_min + x_max) / 2]

	m = folium.Map(location=map_center, zoom_start=15)
	sim_geo = gpd.GeoSeries(selected_polygon).simplify(tolerance=0.001)
	geo_j = sim_geo.to_json()
	geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillOpacity": "0%"})
	folium.Popup(i).add_to(geo_j)
	geo_j.add_to(m)
	heatmap_data = [[row['Lat'], row['Lon'], row['PCA_1']] for idx, row in selected_data.iterrows() if selected_polygon.contains(Point(row['Lon'], row['Lat']))]
	# heatmap_data = [[row['Lat'], row['Lon'], row['PCA_1']] for idx, row in selected_data.iterrows()]
	HeatMap(heatmap_data).add_to(m)
	m.save(f"heatmaps/heatmap_{selected_village}.html")